In [ ]:
import tensorflow as tf
from keras import layers, Model
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.preprocessing import image_dataset_from_directory
import numpy as np
import pandas as pd

#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# Load image data
ds_train = image_dataset_from_directory('/content/gdrive/MyDrive/ECG/Train', batch_size=32, image_size=(224, 224))
ds_test = image_dataset_from_directory('/content/gdrive/MyDrive/ECG/Test', batch_size=32, image_size=(224, 224))

# Load numeric array data
X = np.load('/content/gdrive/MyDrive/ECG/items_AF&SR.npy')
y = np.load('/content/gdrive/MyDrive/ECG/arr2D_AF&SR_label.npy')

# Load metadata
metadata = pd.read_csv('/content/gdrive/MyDrive/ECG/Matadata_ECG.csv')

Found 2869 files belonging to 2 classes.
Found 718 files belonging to 2 classes.


In [ ]:
metadata.head(5)

,Location,Labels
0,/ECG Dataset/ECG/Test/AF/0.png,1
1,/ECG Dataset/ECG/Train/SR/1.png,0
2,/ECG Dataset/ECG/Train/SR/2.png,0
3,/ECG Dataset/ECG/Train/AF/3.png,1
4,/ECG Dataset/ECG/Train/SR/4.png,0


In [ ]:
# Create a new column 'Location_final' by adjusting 'Location'
metadata['Location_final'] = metadata['Location'].apply(lambda x: x.replace('/ECG Dataset/ECG/', '/content/gdrive/MyDrive/ECG/'))

# Drop the original 'Location' column if needed
metadata.drop(columns=['Location'], inplace=True)

# Rename the new column to 'Location' if needed
metadata.rename(columns={'Location_final': 'Location'}, inplace=True)

In [ ]:
metadata.head(5)

,Labels,Location
0,1,/content/gdrive/MyDrive/ECG/Test/AF/0.png
1,0,/content/gdrive/MyDrive/ECG/Train/SR/1.png
2,0,/content/gdrive/MyDrive/ECG/Train/SR/2.png
3,1,/content/gdrive/MyDrive/ECG/Train/AF/3.png
4,0,/content/gdrive/MyDrive/ECG/Train/SR/4.png


In [ ]:
# Preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

print(X_scaled)

[[[ 0.80705828 -0.4588596  -0.25228113]
  [ 0.80705828 -0.4588596  -0.25228113]
  [ 0.80705828 -0.4588596  -0.25228113]
  ...
  [-0.71186622 -0.24757386 -0.22215566]
  [-0.71770824 -0.23683052 -0.22215566]
  [-0.7060242  -0.22608717 -0.22215566]]

 [[-0.7410763  -0.17237046 -0.10165378]
  [-0.7410763  -0.17237046 -0.10165378]
  [-0.7410763  -0.17237046 -0.10165378]
  ...
  [ 0.035912    0.08546976 -0.07152831]
  [ 0.04759603  0.07830753 -0.06550321]
  [ 0.04759603  0.07472642 -0.05646557]]

 [[-2.72736219 -1.55109943 -2.18031125]
  [-2.72736219 -1.55109943 -2.18031125]
  [-2.72736219 -1.55109943 -2.18031125]
  ...
  [-0.35550316  0.38628336  1.4076323 ]
  [-0.41976535  0.38270224  1.39558212]
  [-0.45481746  0.38270224  1.36846919]]

 ...

 [[ 0.16443638 -0.24399274  0.16947546]
  [ 0.16443638 -0.24399274  0.16947546]
  [ 0.16443638 -0.24399274  0.16947546]
  ...
  [-0.05171826 -0.13655932 -0.06550321]
  [-0.08677037 -0.03270701 -0.05947812]
  [-0.0984544   0.03533416 -0.07454085]]

 [

In [ ]:
import tensorflow as tf
from keras import layers, Model
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import pandas as pd



def load_images(file_paths):
    images = []
    for path in file_paths:
        img = load_img(path, target_size=(224, 224))
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)


# Load image data using the adjusted metadata DataFrame
X_train_images = load_images(metadata[metadata['Labels'] == 0]['Location'])
X_test_images = load_images(metadata[metadata['Labels'] == 1]['Location'])

# Define VGG16 base model
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze VGG16 layers
for layer in vgg_base.layers:
    layer.trainable = False

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
vgg_output = layers.GlobalAveragePooling2D()(vgg_base.output)

# Define CNN-bidirectional-LSTM model for numeric array data
model_numeric = tf.keras.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=(1000, 3)),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(256, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.GlobalAveragePooling1D(),
    layers.Dense(64, activation='relu')
])


58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
# Ensure consistent number of samples
num_samples = min(len(metadata), len(X_scaled), len(y))
metadata = metadata.iloc[:num_samples]
X_scaled = X_scaled[:num_samples]
y = y[:num_samples]

# Split data into train and test sets
test_indices = np.arange(4, num_samples, 5)  # Every 5th index starting from the 5th
train_indices = np.setdiff1d(np.arange(num_samples), test_indices)

X_train_images = load_images(metadata.iloc[train_indices]['Location'])
X_train_numeric = X_scaled[train_indices]
y_train = y[train_indices]

X_test_images = load_images(metadata.iloc[test_indices]['Location'])
X_test_numeric = X_scaled[test_indices]
y_test = y[test_indices]

In [ ]:
print(len(X_train_images))
print(len(X_train_numeric))
print(len(X_test_images))
print(len(X_test_numeric))
print(len(y_train))
print(len(y_test))

2870
2870
717
717
2870
717


In [ ]:
# Define fusion layer to combine outputs of VGG16 and CNN-bidirectional-LSTM
fusion_layer = layers.Concatenate()([vgg_output, model_numeric.output])

# Add attention layer
attention_probs = layers.Dense(1, activation='tanh')(fusion_layer)
attention_probs = layers.Flatten()(attention_probs)
attention_probs = layers.Activation('softmax')(attention_probs)
attention_probs = layers.RepeatVector(64)(attention_probs)
attention_probs = layers.Permute([2, 1])(attention_probs)

attention_probs = layers.Reshape((-1, 64))(attention_probs)

# Reshape fusion_layer to match the shape of attention_probs
fusion_layer_reshaped = layers.Reshape((-1, 1, 64))(fusion_layer)

# Perform element-wise multiplication
sent_representation = layers.Multiply()([fusion_layer_reshaped, attention_probs])

# Reshape the result to the desired shape
sent_representation = layers.Reshape((-1, 64))(sent_representation)

# Sum along the appropriate axis
sent_representation = layers.Lambda(lambda xin: tf.keras.backend.sum(xin, axis=-2), output_shape=(64,))(sent_representation)
# Add classification layer
output_layer = layers.Dense(1, activation='sigmoid')(sent_representation)

# Create multimodal model
multimodal_model = Model(inputs=[vgg_base.input, model_numeric.input], outputs=output_layer)

# Compile the model
multimodal_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using pre-split datasets
multimodal_model.fit([X_train_images, X_train_numeric], y_train, epochs=80)

# Evaluate the model using pre-split test datasets
loss, accuracy = multimodal_model.evaluate([X_test_images, X_test_numeric], y_test)
print("Test Accuracy:", accuracy)

Epoch 1/80
90/90 [==============================] - 18s 142ms/step - loss: 3.7426 - accuracy: 0.6641
Epoch 2/80
90/90 [==============================] - 13s 143ms/step - loss: 0.5183 - accuracy: 0.8624
Epoch 3/80
90/90 [==============================] - 13s 146ms/step - loss: 0.3621 - accuracy: 0.9091
Epoch 4/80
90/90 [==============================] - 13s 146ms/step - loss: 0.2204 - accuracy: 0.9383
Epoch 5/80
90/90 [==============================] - 13s 143ms/step - loss: 0.2147 - accuracy: 0.9415
Epoch 6/80
90/90 [==============================] - 13s 144ms/step - loss: 0.1370 - accuracy: 0.9631
Epoch 7/80
90/90 [==============================] - 13s 142ms/step - loss: 0.1389 - accuracy: 0.9561
Epoch 8/80
90/90 [==============================] - 13s 145ms/step - loss: 0.1170 - accuracy: 0.9641
Epoch 9/80
90/90 [==============================] - 13s 146ms/step - loss: 0.0973 - accuracy: 0.9683
Epoch 10/80
90/90 [==============================] - 13s 145ms/step - loss: 0.0764 - accura

In [ ]:
# Define fusion layer to combine outputs of VGG16 and CNN-bidirectional-LSTM
fusion_layer = layers.Concatenate()([vgg_output, model_numeric.output])

# Add attention layer
attention_probs = layers.Dense(1, activation='tanh')(fusion_layer)
attention_probs = layers.Flatten()(attention_probs)
attention_probs = layers.Activation('softmax')(attention_probs)
attention_probs = layers.RepeatVector(64)(attention_probs)
attention_probs = layers.Permute([2, 1])(attention_probs)

attention_probs = layers.Reshape((-1, 64))(attention_probs)

# Reshape fusion_layer to match the shape of attention_probs
fusion_layer_reshaped = layers.Reshape((-1, 1, 64))(fusion_layer)

# Perform element-wise multiplication
sent_representation = layers.Multiply()([fusion_layer_reshaped, attention_probs])

# Reshape the result to the desired shape
sent_representation = layers.Reshape((-1, 64))(sent_representation)

# Sum along the appropriate axis
sent_representation = layers.Lambda(lambda xin: tf.keras.backend.sum(xin, axis=-2), output_shape=(64,))(sent_representation)
# Add classification layer
output_layer = layers.Dense(1, activation='sigmoid')(sent_representation)

# Create multimodal model
multimodal_model = Model(inputs=[vgg_base.input, model_numeric.input], outputs=output_layer)

# Compile the model
multimodal_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using pre-split datasets and store the history
history = multimodal_model.fit([X_train_images, X_train_numeric], y_train, epochs=100, validation_data=([X_test_images, X_test_numeric], y_test))

# Evaluate the model using pre-split test datasets
loss, test_accuracy = multimodal_model.evaluate([X_test_images, X_test_numeric], y_test)
print("Test Accuracy:", test_accuracy)

# Retrieve the train accuracy from the history
train_accuracy = history.history['accuracy'][-1]
print("Train Accuracy:", train_accuracy)

Epoch 1/100
90/90 [==============================] - 45s 335ms/step - loss: 1.8428 - accuracy: 0.6425 - val_loss: 1.5991 - val_accuracy: 0.6890
Epoch 2/100
90/90 [==============================] - 18s 205ms/step - loss: 1.2937 - accuracy: 0.6916 - val_loss: 0.9873 - val_accuracy: 0.7350
Epoch 3/100
90/90 [==============================] - 18s 203ms/step - loss: 0.9583 - accuracy: 0.7286 - val_loss: 0.8086 - val_accuracy: 0.7685
Epoch 4/100
90/90 [==============================] - 16s 176ms/step - loss: 0.7879 - accuracy: 0.7505 - val_loss: 0.6890 - val_accuracy: 0.7727
Epoch 5/100
90/90 [==============================] - 18s 201ms/step - loss: 0.5952 - accuracy: 0.8045 - val_loss: 0.4717 - val_accuracy: 0.8466
Epoch 6/100
90/90 [==============================] - 18s 201ms/step - loss: 0.4629 - accuracy: 0.8557 - val_loss: 0.4043 - val_accuracy: 0.8759
Epoch 7/100
90/90 [==============================] - 16s 178ms/step - loss: 0.3721 - accuracy: 0.8774 - val_loss: 0.3475 - val_accuracy: